In [ ]:
# TODO
# Set up spark session ✔
# Import single-line JSON files ✔
    # Optimize ==> Create a Function to Import all of these files and assign names ot them; perhaps using wrappers?

# Data validation
    # Check if any null values
    # Customer according to specs:




## Required Imports
---

In [3]:
import pyspark 
import pyspark.pandas as pd
from pyspark.sql import SparkSession


c:\Users\GzuzC\OneDrive\Documents\Notes\Personal Studies\Workbooks and Worksheets\Python\PerScholas Data Engineering\perscholas_capstone_project\ignore\ETL-capstone-env\lib\site-packages\pyspark\pandas\__init__.py:49: UserWarning: 'PYARROW_IGNORE_TIMEZONE' environment variable was not set. It is required to set this environment variable to '1' in both driver and executor sides if you use pyarrow>=2.0.0. pandas-on-Spark will set it for you but it does not work if there is a Spark context already launched.
  warnings.warn(


In [4]:
session = SparkSession.builder.appName('Credit Card ETL').getOrCreate()


In [5]:
session

## Extract:
---
- Customer
- Branch
- Credit


In [6]:
customer_df = session.read.load('../data/credit_card_dataset/cdw_sapp_custmer.json', format='json')

In [40]:
branch_df = session.read.load('../data/credit_card_dataset/cdw_sapp_branch.json', format='json')

In [41]:
credit_df = session.read.load('../data/credit_card_dataset/cdw_sapp_credit.json', format='json')

In [42]:
customer_df_dirty_backup = customer_df
branch_df_dirty_backup = branch_df
credit_df_dirty_backup = credit_df

## Transform
---
1. Data Validation
- Ensure Correct Schema, View Current DataTypes
- Determine if any null values

In [50]:
print("Customer DataFrame:\n")
customer_df.printSchema()
print("\nBranch DataFrame:\n")
branch_df.printSchema()
print("\nCredit DataFrame:\n")
branch_df.printSchema()




Customer DataFrame:

root
 |-- APT_NO: string (nullable = true)
 |-- CREDIT_CARD_NO: string (nullable = true)
 |-- CUST_CITY: string (nullable = true)
 |-- CUST_COUNTRY: string (nullable = true)
 |-- CUST_EMAIL: string (nullable = true)
 |-- CUST_PHONE: long (nullable = true)
 |-- CUST_STATE: string (nullable = true)
 |-- CUST_ZIP: string (nullable = true)
 |-- FIRST_NAME: string (nullable = true)
 |-- LAST_NAME: string (nullable = true)
 |-- LAST_UPDATED: string (nullable = true)
 |-- MIDDLE_NAME: string (nullable = true)
 |-- SSN: long (nullable = true)
 |-- STREET_NAME: string (nullable = true)


Branch DataFrame:

root
 |-- BRANCH_CITY: string (nullable = true)
 |-- BRANCH_CODE: long (nullable = true)
 |-- BRANCH_NAME: string (nullable = true)
 |-- BRANCH_PHONE: string (nullable = true)
 |-- BRANCH_STATE: string (nullable = true)
 |-- BRANCH_STREET: string (nullable = true)
 |-- BRANCH_ZIP: long (nullable = true)
 |-- LAST_UPDATED: string (nullable = true)


Credit DataFrame:

root

In [ ]:
def null_checker(dataframe):
    original_count= dataframe.count()
    drop_na = dataframe.dropna()
    new_count = drop_na.count()
    if new_count != original_count:
        print(f"Original Number of Rows: {original_count} \
            \nNew Number of Rows: {new_count}")
    else:
        print("No nulls, no changes made")


In [34]:
# grab column head 
# attach .isNull() 
customer_df_dropped = customer_df.dropna()


In [36]:
customer_df_dropped.count()

952

In [20]:
customer_df.count()

952

In [19]:
customer_df.createOrReplaceTempView("customer_df")

In [39]:
session.sql(f"""
SELECT *
FROM customer_df """)

DataFrame[APT_NO: string, CREDIT_CARD_NO: string, CUST_CITY: string, CUST_COUNTRY: string, CUST_EMAIL: string, CUST_PHONE: bigint, CUST_STATE: string, CUST_ZIP: string, FIRST_NAME: string, LAST_NAME: string, LAST_UPDATED: string, MIDDLE_NAME: string, SSN: bigint, STREET_NAME: string]